### Задание  
На вебинаре мы говорили что долгое время CNN и RNN архитектуры были конкурирующими, выяснить какая архитектура больше подходит для задачи сентимент анализа на данных с вебинара

1. построить свёрточную архитектуру
2. построить различные архитектуры с RNN
3. построить совместные архитектуры CNN -> RNN и/или (RNN -> CNN)
4. сделать выводы что получилось лучше

In [1]:
import numpy as np
import pandas as pd
import re

import keras

from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Masking
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

### Чтение датасета

In [2]:
df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")
df_val = pd.read_csv("data/val.csv")

In [3]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


### Предобработка

In [4]:
def num_replace(word):
    newtoken = 'x' * len(word)
    return newtoken


def clean_token(token, misc):
    out_token = token.strip().replace(' ', '')
    if token == 'Файл' and 'SpaceAfter=No' in misc:
        return None
    return out_token


def clean_lemma(lemma, pos):
    out_lemma = lemma.strip().replace(' ', '').replace('_', '').lower()
    if '|' in out_lemma or out_lemma.endswith('.jpg') or out_lemma.endswith('.png'):
        return None
    if pos != 'PUNCT':
        if out_lemma.startswith('«') or out_lemma.startswith('»'):
            out_lemma = ''.join(out_lemma[1:])
        if out_lemma.endswith('«') or out_lemma.endswith('»'):
            out_lemma = ''.join(out_lemma[:-1])
        if out_lemma.endswith('!') or out_lemma.endswith('?') or out_lemma.endswith(',') \
                or out_lemma.endswith('.'):
            out_lemma = ''.join(out_lemma[:-1])
    return out_lemma

In [5]:
def process(pipeline, text='Строка', keep_pos=True, keep_punct=False):
    entities = {'PROPN'}
    named = False
    memory = []
    mem_case = None
    mem_number = None
    tagged_propn = []

    # обрабатываем текст, получаем результат в формате conllu:
    processed = pipeline.process(text)

    # пропускаем строки со служебной информацией:
    content = [l for l in processed.split('\n') if not l.startswith('#')]

    # извлекаем из обработанного текста леммы, тэги и морфологические характеристики
    tagged = [w.split('\t') for w in content if w]

    for t in tagged:
        if len(t) != 10:
            continue
        (word_id, token, lemma, pos, xpos, feats, head, deprel, deps, misc) = t
        token = clean_token(token, misc)
        lemma = clean_lemma(lemma, pos)
        if not lemma or not token:
            continue
        if pos in entities:
            if '|' not in feats:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            morph = {el.split('=')[0]: el.split('=')[1] for el in feats.split('|')}
            if 'Case' not in morph or 'Number' not in morph:
                tagged_propn.append('%s_%s' % (lemma, pos))
                continue
            if not named:
                named = True
                mem_case = morph['Case']
                mem_number = morph['Number']
            if morph['Case'] == mem_case and morph['Number'] == mem_number:
                memory.append(lemma)
                if 'SpacesAfter=\\n' in misc or 'SpacesAfter=\s\\n' in misc:
                    named = False
                    past_lemma = '::'.join(memory)
                    memory = []
                    tagged_propn.append(past_lemma + '_PROPN ')
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN ')
                tagged_propn.append('%s_%s' % (lemma, pos))
        else:
            if not named:
                if pos == 'NUM' and token.isdigit():  # Заменяем числа на xxxxx той же длины
                    lemma = num_replace(token)
                tagged_propn.append('%s_%s' % (lemma, pos))
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN ')
                tagged_propn.append('%s_%s' % (lemma, pos))

    if not keep_punct:
        tagged_propn = [word for word in tagged_propn if word.split('_')[1] != 'PUNCT']
    if not keep_pos:
        tagged_propn = [word.split('_')[0] for word in tagged_propn]
    return tagged_propn

In [6]:
from ufal.udpipe import Model, Pipeline
import wget
import os
import sys

udpipe_model_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
udpipe_filename = udpipe_model_url.split('/')[-1]

if not os.path.isfile(udpipe_filename):
    print('UDPipe model not found. Downloading...', file=sys.stderr)
    wget.download(udpipe_model_url)

print('\nLoading the model...', file=sys.stderr)
model = Model.load(udpipe_filename)
process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')


Loading the model...


In [7]:
def preprocess_text(txt, keep_pos):
    txt = str(txt)
    txt = re.sub("@[\w]*", ' ', txt)
    output = process(process_pipeline, text=txt, keep_pos=keep_pos)
    return ' '.join(output)

In [8]:
preprocess_text(df_train['text'][0], keep_pos=False)

'не уезжаааааааать ❤ я тоже не хотеть чтобы ты уезжать'

In [9]:
%%time
df_train['text'] = df_train['text'].apply(preprocess_text, keep_pos=False)
df_val['text'] = df_val['text'].apply(preprocess_text, keep_pos=False)
df_test['text'] = df_test['text'].apply(preprocess_text, keep_pos=False)

Wall time: 31min 10s


In [10]:
df_train.head()

,id,text,class
0,0,не уезжаааааааать ❤ я тоже не хотеть чтобы ты ...,0
1,1,rt ребята и девчата все в кино вот это любовь ...,1
2,2,rt кто ненавидеть пробка ретвит rt,0
3,3,rt хотеться котлета по-киевска запретный плод,1
4,4,босапоп есбоса не бояться мороз и,1


### Векторизуем

In [11]:
training_length = max([len(i.split()) for i in df_train['text'].values])
training_length

35

In [12]:
vectorizer = TextVectorization(standardize=None, max_tokens=20000, output_sequence_length=training_length)
vectorizer.adapt(df_train['text'].values)

In [13]:
X_train = vectorizer(np.array([[s] for s in df_train['text'].values])).numpy()
X_valid = vectorizer(np.array([[s] for s in df_val['text'].values])).numpy()

In [14]:
voc = vectorizer.get_vocabulary()
word_count = len(voc) + 2

In [15]:
y_train = df_train['class'].values
y_val = df_val['class'].values

### Моделируем

In [16]:
early_stopping=EarlyStopping(monitor='val_loss')

#### RNN

##### SimpleRNN

In [17]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [18]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 14s 44ms/step - loss: 0.5620 - accuracy: 0.7037 - val_loss: 0.4990 - val_accuracy: 0.7500
Epoch 2/10
319/319 [==============================] - 14s 44ms/step - loss: 0.4610 - accuracy: 0.7827 - val_loss: 0.5057 - val_accuracy: 0.7440


In [19]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 11ms/step - loss: 0.5129 - accuracy: 0.7381


Test loss: 0.5128977298736572
Test accuracy: 0.7380858063697815


##### LSTM

In [20]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [21]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 30s 96ms/step - loss: 0.5587 - accuracy: 0.6953 - val_loss: 0.4880 - val_accuracy: 0.7522
Epoch 2/10
319/319 [==============================] - 30s 93ms/step - loss: 0.4669 - accuracy: 0.7754 - val_loss: 0.4866 - val_accuracy: 0.7498
Epoch 3/10
319/319 [==============================] - 30s 94ms/step - loss: 0.4411 - accuracy: 0.7901 - val_loss: 0.4901 - val_accuracy: 0.7581


In [22]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 21ms/step - loss: 0.4948 - accuracy: 0.7560


Test loss: 0.4948217272758484
Test accuracy: 0.7560287714004517


#### GRU

In [23]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(GRU(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [24]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 26s 80ms/step - loss: 0.6931 - accuracy: 0.5050 - val_loss: 0.6934 - val_accuracy: 0.5049
Epoch 2/10
319/319 [==============================] - 25s 79ms/step - loss: 0.6932 - accuracy: 0.5055 - val_loss: 0.6932 - val_accuracy: 0.5049
Epoch 3/10
319/319 [==============================] - 25s 80ms/step - loss: 0.6931 - accuracy: 0.5070 - val_loss: 0.6931 - val_accuracy: 0.5049
Epoch 4/10
319/319 [==============================] - 26s 81ms/step - loss: 0.6931 - accuracy: 0.5075 - val_loss: 0.6931 - val_accuracy: 0.5049


In [25]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 20ms/step - loss: 0.6931 - accuracy: 0.5047


Test loss: 0.693148136138916
Test accuracy: 0.5047392249107361


#### CNN

In [26]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
    
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [27]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 10s 32ms/step - loss: 0.5913 - accuracy: 0.6897 - val_loss: 0.5114 - val_accuracy: 0.7539
Epoch 2/10
319/319 [==============================] - 11s 34ms/step - loss: 0.4971 - accuracy: 0.7764 - val_loss: 0.4896 - val_accuracy: 0.7637
Epoch 3/10
319/319 [==============================] - 11s 34ms/step - loss: 0.4521 - accuracy: 0.8053 - val_loss: 0.4888 - val_accuracy: 0.7636
Epoch 4/10
319/319 [==============================] - 11s 34ms/step - loss: 0.4101 - accuracy: 0.8299 - val_loss: 0.5121 - val_accuracy: 0.7661


In [28]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 0s 6ms/step - loss: 0.5145 - accuracy: 0.7596


Test loss: 0.5144764184951782
Test accuracy: 0.7596437931060791


#### CNN + RNN

In [32]:
from tensorflow.keras.layers import Reshape, Permute

In [33]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))

# Reshape with the first argument being the number of filter in your last conv layer
model.add(Reshape((10, -1)))
# Just write this Permute after, its complicated why
model.add(Permute((2, 1)))

model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

    
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [34]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 12s 36ms/step - loss: 0.5466 - accuracy: 0.7082 - val_loss: 0.4769 - val_accuracy: 0.7618
Epoch 2/10
319/319 [==============================] - 11s 36ms/step - loss: 0.4348 - accuracy: 0.7976 - val_loss: 0.4734 - val_accuracy: 0.7668
Epoch 3/10
319/319 [==============================] - 11s 35ms/step - loss: 0.3682 - accuracy: 0.8376 - val_loss: 0.4977 - val_accuracy: 0.7665


In [35]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 0s 7ms/step - loss: 0.5035 - accuracy: 0.7641


Test loss: 0.5034821033477783
Test accuracy: 0.7640523910522461


__ВЫВОД:__ Наблюдаем, что из всех рассмотренных моделей комбинация архитектур CNN + RNN показывает лучший результат